In [1]:
EXPERIMENT_IDENTIFIER = "framing_with_llm_paper_prompt"

In [2]:
N = 109

In [3]:
!ls ../data/sharif_frame_annotation.csv

../data/sharif_frame_annotation.csv


In [4]:
import pandas as pd
import random
random.seed(42)

sample_df = pd.read_csv("../data/sharif_frame_annotation.csv")
sample_df.head()

,News Articles,Formal Frame,Sub Frame (Optional)
0,বিভিন্ন উদ্ভাবনী উদ্যোগকে উৎসাহিত করতে এবং সরক...,Attribution of responsibility.,Economic
1,কক্সবাজারের দ্বিতীয় বৃহত্তম পর্যটন স্পট ইনানী ...,Economic,Human interest
2,<p>ইতালিতে অবস্থানরত প্রবাসী বাংলাদেশিদের পাঠা...,Economic,Human interest
3,"উপজাতিদের সাংবিধানিক স্বীকৃতি, শিক্ষা ও চাকরির...",Human interest,Conflict
4,রাঙ্গামাটির কাউখালী উপজেলার দুর্গম পানছড়িতে যৌ...,Conflict,Attribution of responsibility.


In [6]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

### Loading model

In [7]:
pip install openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [16]:
from openai import OpenAI
import json

client = OpenAI()

def get_response(news_article):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": news_article}
        ]
    )
    return completion.choices[0].message.content

In [26]:
with open('framing_prompt_llm_paper_plus_our_frames.txt', 'r') as file:
    prompt = file.read()
print(prompt)

Decide the text's frame between these options:

    - responsibility. This frame presents an issue or problem in such a way as to attribute responsibility for its cause or solution to either the government or to an individual or group.
    - human. This frame brings a human face or an emotional angle to the presentation of an event, issue, or problem.
    - conflict. This frame emphasizes conflict between individuals, groups, or institutions as a means of capturing audience interest.
    - morality. This frame puts the event, problem, or issue in the context of religious tenets or moral prescriptions.
    - economic. This frame reports an event, problem, or issue in terms of the consequences it will have economically on an individual, group, institution, region, or country.

Also, assign subframe between these options: 
- Culture
- Violence and Victimization
- Sports & Entertainment
- Nature
- Conflict & Protest
- Crime
- Legal Frameworks
- Local news
- Health
- Education
- Investigati

In [54]:
N = 109
print(f"Selectin {N} articles from {len(articles)} total articles.")

Selectin 10 articles from 109 total articles.


In [55]:
from tqdm import tqdm
import pickle

# Define checkpoint file path
checkpoint_file = 'responses_checkpoint_framing_sharif_sample_4o_try4.pkl'

# Load checkpoint if it exists
try:
    with open(checkpoint_file, 'rb') as f:
        responses = pickle.load(f)
        start_index = len(responses)
        tqdm.write(f"Resuming from checkpoint at index {start_index}")
except FileNotFoundError:
    responses = []
    start_index = 0

# Fetch responses with checkpoints
for i, article in tqdm(enumerate(articles[start_index:N]), total=N - start_index, desc="Processing articles"):
    # Ensure we're only processing from the checkpoint onward
    response = get_response(article[:10000])  # Slice article to handle length limit
    responses.append(response)
    
    # Save checkpoint every 100 responses
    if (i + start_index + 1) % 100 == 0:
        with open(checkpoint_file, 'wb') as f:
            pickle.dump(responses, f)
        tqdm.write(f"Checkpoint saved at index {i + start_index + 1}")

# Save final responses
with open(checkpoint_file, 'wb') as f:
    pickle.dump(responses, f)
tqdm.write("Processing complete and final checkpoint saved.")

Processing articles: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:14<00:00,  1.48s/it]

Processing complete and final checkpoint saved.


In [56]:
responses[:10]

['```json\n{\n  "frame": "responsibility",\n  "subframe": ["Govt. Actions", "Economic Growth and Development"]\n}\n```',
 '```json\n{\n  "frame": "economic",\n  "subframe": ["Conflict & Protest", "Violence and Victimization", "Politics"]\n}\n```',
 '```json\n{\n  "frame": "economic",\n  "subframe": "Economic Growth and Development"\n}\n```',
 '```json\n{\n  "frame": "responsibility",\n  "subframe": ["Empowerment and Advocacy"]\n}\n```',
 '```json\n{\n  "frame": "responsibility",\n  "subframe": ["Crime", "Conflict & Protest"]\n}\n```',
 '```json\n{\n  "frame": "responsibility",\n  "subframe": "Govt. Actions"\n}\n```',
 '```json\n{\n  "frame": "conflict",\n  "subframe": ["Violence and Victimization", "Empowerment and Advocacy"]\n}\n```',
 '```json\n{\n  "frame": "conflict",\n  "subframe": "Sports & Entertainment"\n}\n```',
 '```json\n{\n    "frame": "human",\n    "subframe": "Sports & Entertainment"\n}\n```',
 '```json\n{\n  "frame": "responsibility",\n  "subframe": "Conflict & Protest"\

In [42]:
import joblib
joblib.dump(responses, f"../data/gpt4omini_turbo_{EXPERIMENT_IDENTIFIER}_articles_{N}.joblib")

['../data/gpt4omini_turbo_framing_with_llm_paper_prompt_articles_109.joblib']

In [32]:
joblib.dump(articles, f"../data/gpt4omini_{EXPERIMENT_IDENTIFIER}_responses_{N}.joblib")

['../data/gpt4omini_framing_with_llm_paper_prompt_responses_109.joblib']

In [49]:
import re

def fix_quotes(json_str):
    print(f"Before fixing: {json_str}")
    apostrophe = '"'
    parts = json_str.split(",")
    fixed_parts = []
    for part in parts:
        if "genre" not in part:

            # Detecting error
            count_apostrophe = 0
            for alphabet in part:
                if alphabet == apostrophe:
                    count_apostrophe+=1
            # Fixing
            if count_apostrophe > 4:
                better_part = []
                count_apostrophe = 0
                for alphabet in part:
                    if alphabet == apostrophe:
                        count_apostrophe+=1
                    if count_apostrophe == 4:
                        better_part.append("'")
                        count_apostrophe = 0
                    else:
                        better_part.append(alphabet)
                part = "".join(better_part)
                
            fixed_parts.append(part)
        else:
            fixed_parts.append(part)
        
    temp = ", ".join(fixed_parts)
    print(f"After fixing: {temp}")
    return temp
    
def post_process_chatgpt_turbo_3_5_response_to_json(text):
    #print(f"Before fixing: {text}")

    json_string = text.replace("```", "").replace("json","").replace("\n","").replace("'", '"').replace("\t","").replace('"s ', '\'s ')
    json_string = json_string.replace("subframes", "subframe")
    try:
        json_output = json.loads(json_string)
        return json_output
    except:
        json_output = json.loads(fix_quotes(json_string))
        return json_output

In [50]:
annotated_df = pd.DataFrame(articles)
annotated_df.columns = ["content"]
annotated_df["Frame"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["frame"] for response in responses]
annotated_df["Sub-Frame(s)"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["subframe"] for response in responses]
annotated_df

,content,Frame,Sub-Frame(s)
0,বিভিন্ন উদ্ভাবনী উদ্যোগকে উৎসাহিত করতে এবং সরক...,responsibility,"[Govt. Actions, Economic Growth and Development]"
1,কক্সবাজারের দ্বিতীয় বৃহত্তম পর্যটন স্পট ইনানী ...,responsibility,"[Politics, Conflict & Protest]"
2,<p>ইতালিতে অবস্থানরত প্রবাসী বাংলাদেশিদের পাঠা...,economic,"[Economic Growth and Development, Local news]"
3,"উপজাতিদের সাংবিধানিক স্বীকৃতি, শিক্ষা ও চাকরির...",human,"[Empowerment and Advocacy, Education, Politics]"
4,রাঙ্গামাটির কাউখালী উপজেলার দুর্গম পানছড়িতে যৌ...,responsibility,Crime
...,...,...,...
104,<p>ট্রলার থেকে নদীতে ঝাঁপিয়ে পড়ে অপহরণকারীদের ...,responsibility,"[Crime, Local news]"
105,<p>ফরিদপুরের বোয়ালমারীতে এক তরুণের বিরুদ্ধে নব...,responsibility,Crime
106,বেনাপোলের রঘুনাথপুর সীমান্ত থেকে মঙ্গলবার সকা...,responsibility,"[Crime, Govt. Actions]"
107,জেলার বাঘাইছড়ি উপজেলার তুলাবান এলাকা থেকে গত শ...,conflict,"[Conflict & Protest, Violence and Victimization]"


In [51]:
annotated_df.to_csv("../data/frame_gpt-40-mini_validation_109_v1.csv")

In [24]:
with open(checkpoint_file, 'rb') as f:
    responses = pickle.load(f)

In [38]:
len(responses)

5000

In [40]:
ethnic_sample_df = pd.DataFrame(articles)
ethnic_sample_df.columns = ["content"]
ethnic_sample_df

,content
0,<p>ফকিরহাট উপজেলা স্বাস্থ্য ও পরিবার পরিকল্পনা...
1,ব্লু-হোয়েল শেষে অতঃপর চলছে মোমো আতঙ্ক। তবে মোম...
2,"<p>এদিকে প্রযোজক জানিয়েছেন, ছবিটির জন্য প্রচার..."
3,"<p class=""Hdg1col"">প্রথমবারের মতো দক্ষিণ আফ্রি..."
4,<p>মফস্বলের শিক্ষার্থীদের ইংরেজি–ভীতি দূর করতে...
...,...
4995,জন্মের দেড় বছর পেরিয়ে গেলেও শরীরে অস্ত্রোপচার ...
4996,"<p>শিক্ষামন্ত্রী নুরুল ইসলাম নাহিদ বলেছেন, প্র..."
4997,"<p class=""TEXT"">দেশে করোনা শনাক্তকরণ পরীক্ষা শ..."
4998,বিএনপির ভারপ্রাপ্ত মহাসচিব মির্জা ফখরুল ইসলাম ...


In [75]:
ethnic_sample_df["Genre"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["genre"] for response in responses]
ethnic_sample_df["Journalism Style"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["journalism_style"] for response in responses]
ethnic_sample_df["Featured"] = [ post_process_chatgpt_turbo_3_5_response_to_json(response).get('featured', pd.NA) for response in responses ]
ethnic_sample_df["Is_ethnic"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["is_indigenous"] for response in responses]
ethnic_sample_df["Sentiment"] = [ post_process_chatgpt_turbo_3_5_response_to_json(response).get('sentiment', pd.NA) for response in responses ]
ethnic_sample_df["Agenda Setting"] = [ post_process_chatgpt_turbo_3_5_response_to_json(response).get('agenda_setting', pd.NA) for response in responses ]
ethnic_sample_df["Framing"] = [ post_process_chatgpt_turbo_3_5_response_to_json(response).get('framing', pd.NA) for response in responses ]
ethnic_sample_df["Priming"] = [ post_process_chatgpt_turbo_3_5_response_to_json(response).get('priming', pd.NA) for response in responses ]
ethnic_sample_df["Mobilizing"] = [ post_process_chatgpt_turbo_3_5_response_to_json(response).get('mobilizing', pd.NA) for response in responses ]

ethnic_sample_df

,content,Genre,Journalism Style,Is_ethnic,Featured,Sentiment,Agenda Setting,Framing,Priming,Mobilizing
0,<p>ফকিরহাট উপজেলা স্বাস্থ্য ও পরিবার পরিকল্পনা...,[Local news],Straight,False,Subject,Negative,Health,Tragedy,Awareness,Support
1,ব্লু-হোয়েল শেষে অতঃপর চলছে মোমো আতঙ্ক। তবে মোম...,[Crime],Straight,False,Subject,Negative,Online Games,Dangerous Influence,Youth,Awareness
2,"<p>এদিকে প্রযোজক জানিয়েছেন, ছবিটির জন্য প্রচার...",[Culture & Entertainment],Straight,False,Not applicable,Neutral,Promotion,Promotion,Advertisement,None
3,"<p class=""Hdg1col"">প্রথমবারের মতো দক্ষিণ আফ্রি...",[Sports],Straight,False,Subject,Positive,Sports,Competition,Victory,Celebration
4,<p>মফস্বলের শিক্ষার্থীদের ইংরেজি–ভীতি দূর করতে...,[Education],Straight,False,Subject,Negative,English Language,Competition,Students,Improvement
...,...,...,...,...,...,...,...,...,...,...
4995,জন্মের দেড় বছর পেরিয়ে গেলেও শরীরে অস্ত্রোপচার ...,"[Health, Local News]",Straight,False,Personal Experience,Positive,Treatment,Family Issues,Healthcare,Community Support
4996,"<p>শিক্ষামন্ত্রী নুরুল ইসলাম নাহিদ বলেছেন, প্র...","[Politics, Education]",Straight,False,Subject,Neutral,Education,Policy,Feedback,Reform
4997,"<p class=""TEXT"">দেশে করোনা শনাক্তকরণ পরীক্ষা শ...",[Health],Straight,False,Expert or Commentator,Neutral,Testing,Prevention,Healthcare,Awareness
4998,বিএনপির ভারপ্রাপ্ত মহাসচিব মির্জা ফখরুল ইসলাম ...,"[Politics, Crime]",Straight,False,Subject,Negative,Legal,Controversy,Court,Protest


In [76]:
ethnic_sample_df.isna().sum()

content               0
Genre                 0
Journalism Style     44
Is_ethnic             1
Featured            972
Sentiment            81
Agenda Setting       73
Framing              72
Priming              75
Mobilizing           78
dtype: int64

In [77]:
ethnic_sample_df.to_csv(f"../data/nonethnic_{N}_gpt_3.5_turbo_v3.csv")

### Evaluating

In [25]:
true_labels = [1,1,0,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,1,1,0,0,1,0,1,0,1,1,0,1,0,0,1,0,1,1,1,1,0,1,1,1,0,1,1,1,0,
               1,1,1,0,1,0,0,0,1,1,1,1,0,1,0,1,
               0,1,0,1,1,1,1,0,1,1,1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,1,1,1,1,0,1,1,1,1,0,1]

In [27]:
predicted_labels = ethnic_sample_df["Is_ethnic"].values.tolist()
predicted_labels = [1 if item == True else 0 for item in predicted_labels]
predicted_labels[:10]

[1, 0, 1, 0, 1]

In [28]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

# Calculating precision, recall, accuracy, and F1 score
precision = precision_score(true_labels[:N], predicted_labels)
recall = recall_score(true_labels[:N], predicted_labels)
accuracy = accuracy_score(true_labels[:N], predicted_labels)
f1 = f1_score(true_labels[:N], predicted_labels)

# Print the results
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Precision: 0.6666666666666666
Recall: 0.6666666666666666
Accuracy: 0.6
F1 Score: 0.6666666666666666


In [108]:
ethnic_sample_df["Is_ethnic"].values.tolist()

[True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True]

### MISC

In [11]:
abhi_labels = [1,1,0,0,1,1,1,0,1,1,0,0,1,1,1,1,1,1,0,
               1,1,0,0,1,0,1,0,1,1,0,1,1,0,0,0,1,1,1,
               1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,1,0,1,1,
               1,1,0,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,
               0,0,1,0,1,
               1,0,0,0,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,]
sharif_labels = [ 1,1,0,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,1,1,1,0,
                 1,1,1,0,1,0,0,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,1,1,0,1,1,1,1,0,1 ]

In [21]:
import numpy as np
def annotator_agreement_score(labels1, labels2):
    # Calculate agreement score
    total_labels = len(labels1)
    agreements = np.sum(np.array(labels1) == np.array(labels2))
    agreement_score = agreements / total_labels

    # Find conflicting posts
    conflict_indices = [i for i in range(total_labels) if labels1[i] != labels2[i]]
    conflicting_posts = [(i, labels1[i], labels2[i]) for i in conflict_indices]

    return agreement_score, len(conflicting_posts)
annotator_agreement_score(abhi_labels, sharif_labels)

NameError: name 'abhi_labels' is not defined

In [80]:
from collections import Counter

counts = Counter(true_labels)
print("Yes count:", counts[1])
print("No count:", counts[0])

Yes count: 63
No count: 37


In [45]:
ethnic_sample_df.iloc[2].values.tolist()

[np.int64(2),
 'গাইবান্ধার গোবিন্দগঞ্জের রংপুর চিনিকলের সাহেবগঞ্জ ইক্ষু খামারে দেয়া কাঁটাতারের বেড়ার পিলার ভাংচুর করেছে দুর্বৃত্তরা।সোমবার রাতে খামার সংলগ্ন জয়পুর পাড়ার দক্ষিণ অংশে দেয়া ২৬টি পিলার ভাংচুর করা হয়।খামারের জমি সাঁওতালদের কাছ থেকে দখলমুক্ত করার পর মিল কর্তৃপক্ষ এই কাঁটাতারের বেড়া দেয়। এরপর সোমবার রাতে সাঁওতাল পল্লী সংলগ্ন খামারের বেড়ার পিলার ভাংচুর করে দুর্বৃত্তরা।রংপুর সুগার মিলের ব্যবস্থাপনা পরিচালক আব্দুল আউয়াল জানান, দুস্কৃতকারীরা রাতের আধারে খামারের পশ্চিম অংশে সাঁওতাল পল্লী জয়পুর পাড়া সংলগ্ন ২৬টি পিলার ভাংচুর করে। এ ঘটনায় ১৩ জনের নাম উল্লেখ করে খামারের উপ-ব্যবস্থাপক মো. হুমায়ন কবির মঙ্গলবার রাতে গোবিন্দগঞ্জ থানায় একটি জিডি করেন।গোবিন্দগঞ্জ থানার ওসি সুব্রত কুমার সরকার বিষয়টি নিশ্চিত করে বলেন, ঘটনার বিষয়টি খতিয়ে দেখা হচ্ছে।গত ৬ নভেম্বর আদিবাসী সাঁওতালদের সঙ্গে আইনশৃংখলা বাহিনী চিনিকলের শ্রমিক-কর্মচারীদের সংঘর্ষের পর ইক্ষু খামার থেকে সাঁওতাল বসতি উচ্ছেদ, লুটপাট, গুলিতে ৩ সাঁওতাল নিহত হওয়ার ঘটনায় সেখানে এখনও অস্থিরতা রয়েছে।',
 'Crime',
 'Investigative',
 'Victim',
 np.Tr

In [78]:
ethnic_sample_df = ethnic_df.sample(n=100, random_state=3)

In [79]:
ethnic_sample_df.to_csv("../data/annotated_100_ethnic_article_against_5_metrics.csv")

### Temp

In [46]:
predicted_sentiment_labels = [1,0,1,1,0,1,0,2,2,0,1,0,2,2,0,1,2,2,0,2,0,2,0,1,0,0,0,0,0,0,0,0,1,2,0,2,0,0,2,1,2,2,2,1,0,1,
                              0,1,0,2,0,1,2,0,0,0,2,1,0,2,1,2,1,1,0,0,2,0,0,1,1,2,1,0,0,2,0,2,1,0,1,2,2,0,0,1,1,0,1,0,0,0,0,
                              2,0,1,1,2,1,1,0,2,2,0,0,0,0,0,2]
sharif_sentiment_labels = [1,0,1,1,1,0,0,0,1,0,0,0,1,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,1,0,0,2,1,0,0,0,1,0,
                           0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,0,1,2,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,2,
                           1,1,0,2,1,0,0,0,0,0,0,]
abhijit_sentiment_labels = [1,0,1,1,0,0,0,0,2,0,2,0,2,2,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,
                            0,0,1,0,0,0,0,0,0,0,0,1,0,2,1,0,0,2,0,0,2,1,1,1,0,0,0,0,0,1,0,1,2,2,0,0,0,1,0,1,0,0,0,2,2,0,1,1,0,
                            2,2,0,2,1,1,0,0,0,0,0,]
true_sentiment_labels = [1,0,1,1,0,0,0,0,1,0,0,0,2,2,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,1,0,0,2,1,0,0,0,1,0,0,
                             0,1,0,0,0,0,0,0,0,0,1,0,2,1,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,0,1,2,2,0,0,0,1,0,1,0,0,0,0,2,0,1,1,2,1,1,0,
                             2,1,1,0,0,0,0,0,]

In [48]:
annotator_agreement_score(abhijit_sentiment_labels, sharif_sentiment_labels)

(np.float64(0.8623853211009175), 15)

In [53]:
from collections import Counter

counts = Counter(true_sentiment_labels)
print("Pos count:", counts[1])
print("Neg count:", counts[0])
print("Neu count:", counts[2])

NameError: name 'true_sentiment_labels' is not defined

In [52]:
counts

NameError: name 'counts' is not defined

In [50]:
N = 109

In [52]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report

accuracy = accuracy_score(true_sentiment_labels, predicted_sentiment_labels)
print(f'Accuracy: {accuracy:.2f}')

# Calculate precision, recall, and F1 score (average='weighted' considers the imbalance in classes)
precision = precision_score(true_sentiment_labels, predicted_sentiment_labels, average='weighted')
recall = recall_score(true_sentiment_labels, predicted_sentiment_labels, average='weighted')
f1 = f1_score(true_sentiment_labels, predicted_sentiment_labels, average='weighted')

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

# Print a detailed classification report
print("\nClassification Report:")
print(classification_report(true_sentiment_labels, predicted_sentiment_labels))

Accuracy: 0.74
Precision: 0.85
Recall: 0.74
F1 Score: 0.77

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.69      0.81        71
           1       0.72      0.84      0.78        25
           2       0.37      0.85      0.51        13

    accuracy                           0.74       109
   macro avg       0.69      0.79      0.70       109
weighted avg       0.85      0.74      0.77       109

